In [1]:
import PyPDF2
import re
import os
from tkinter import filedialog
import tkinter as tk
# import pandas as pd
# import openpyxl as op
import xlwings as xw
from tkinter import simpledialog
import fitz
from fitz import Document, Page, Rect
import os.path
from selenium import webdriver
from selenium.webdriver import ActionChains
from time import sleep
import random
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
file_full = tk.filedialog.askopenfilename()
file_type = os.path.splitext(file_full)[1]
list_path = os.path.dirname(file_full)

In [3]:
wb = xw.Book(file_full)
ws = wb.sheets.active
lr = ws.range('B' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
# pn_list = ws.range('B1:B' + str(lr)).value

In [4]:
driver = webdriver.Edge()
actions = ActionChains(driver)
#docmentation/report/Unrel Part number revision history
edoc_url = "http://rdt.kla-tencor.com/rdt2/edoc_ei/scripts/eiPartRevHistory.asp?txtProduct=EDOC&Tab=4"
driver.get(edoc_url)
print('opening Edoc ...')
edoc_page = driver.current_window_handle
sleep(2)
driver.switch_to.window(edoc_page)




opening Edoc ...


In [8]:
lr = ws.range('C' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
# print('lst row:',lr)
for i in range(2,lr+1):
    mpn = str(ws.range(i,3).value) #.strip()
    driver.switch_to.window(edoc_page)
    search_input = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
        '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form[3]/table/tbody/tr[1]/td[2]/input')))
#     mpn = 'DF-HSL-1524'
    search_input.clear()
    search_input.send_keys(mpn+'*')
    sleep(1)
    search_input.send_keys(Keys.RETURN)
    driver.switch_to.window(driver.window_handles[-1])
    
    table = driver.find_elements(By.XPATH, \
            '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/table[2]/tbody/tr')
    if len(table) < 1:
        driver.switch_to.window(driver.window_handles[-1])
        back_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,\
        '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr[3]/td/div/input')))
        back_btn.click()
        ws.range(i,6).value = 'Not found in EDOC'
    elif len(table) > 2:
        ask = tk.messagebox.askokcancel('Confirm','Please check:')
        print('more than one PNs found')
    else:
        driver.switch_to.window(driver.window_handles[-1])
        print(table[1].text)
        ws.range(i,6).value = table[1].text
        
        
        back_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,\
            '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr/td/div/input')))

        back_btn.click()
    
    

In [17]:
#check on Enovia
enovia_url = 'https://3dx.kla-tencor.com/3dspace/common/KLA_emxNavigator.jsp?ticket=ST-56034-MgBi6B0tOkgnvCEwahkW-cas&collabSpace=KLA+Collabspace'
driver.get(enovia_url)
print('opening Enovia ...')
enovia_page = driver.current_window_handle

opening Enovia ...


In [19]:
lr = ws.range('B' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
# print('lst row:',lr)
for i in range(2,lr+1):
    mpn = str(ws.range(i,2).value) #.strip()
    print(mpn)
    driver.switch_to.window(enovia_page)
    search_field = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
            '//*[@id="quickSearch"]')))
    search_field.clear()
    search_field.send_keys(mpn[:7]+'*')
    sleep(1)
    search_field.send_keys(Keys.RETURN)
    driver.switch_to.window(driver.window_handles[-1])
    driver.switch_to.frame('tableContentFrame')
    driver.switch_to.frame('tableBodyRight')
    affected_items = driver.find_elements(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr')
    for j in range(len(affected_items)):
        dash_num = WebDriverWait(driver, 2).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j+1) + ']/td[18]')))
        if mpn[:11] in affected_items[j].text\
            and dash_num.text != '':
            print(affected_items[j].text)
            des_eno = WebDriverWait(driver, 2).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j+1) + ']/td[10]/span')))
            status_eno = WebDriverWait(driver, 2).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j+1) + ']/td[6]/a')))
            ws.range(i,7).value = des_eno.text
            ws.range(i,8).value = status_eno.text
    driver.close()
#     break

1083822-000
Mechanical Assembly 1083822-000 AA Preliminary No No No KIT,DRAIN,CHILLER,NTHWK SCM SFS Nighthawk Lootens, Jim HALB -000
1077610-000
Mechanical Assembly 1077610-000 AA Preliminary No No No HOSE ASSY, DRAIN CHLR TANKS, NTHWK SCM SFS Nighthawk Mackessy, Heidi HALB -000
426110-01
1079526-000
Pneumatic and Hydraulic Component 1079526-000 AA Preliminary No No No VALVE,ON-OFF,FML-FML,12MM TUBE OD SCM SFS Nighthawk UDAYAKUMAR, BALAJI HAWA -000
0070773-000
Mechanical Design 0070773-000 AA Release No No Yes LABEL, SELF LAMINATING,1IN X3.75IN VINYL 1 SCM SFS SP2 SOGHOYAN, ZAVEN 1001 (20 - Active)
1002 (20 - Active)
HAWA -000 A28477-21 SEND NEW PARTS TO ORACLE Implemented No
Mechanical Design 0070773-000 AB Release No No Yes LABEL,SELF LAMINATING,1IN X3.75IN,VINYL 1 SCM SFS SP2 BALAJI, SRINATH 1001 (20 - Active)
1002 (20 - Active)
HAWA -000 C11591 SP7 IMC AND DATA SAVE ASSY CHANGES Release No
1077611-000
Mechanical Assembly 1077611-000 AA Preliminary No No No HOSE ASSY,ADAPTER,CDA DRA

KeyboardInterrupt: 